In [38]:
import pandas as pd
import pymysql as pymysql

debug = True

start_date = '2016-01-01'
end_date = '2020-01-01'
resample_interval = '5T'

host = "mariadb.mmto.arizona.edu"
database = "measurements"
table = 'bump'
usr = "webuser"
pw = "Clear7Skies!"

conn = pymysql.connect(host=host, port=3306, user=usr, passwd=pw, db=database)

In [39]:
#  This gets the individual e-series thermocouples parameter names
def get_names(connection, search="temptrax"):
    df = None
    try:
        # Read data
        with connection.cursor() as cursor:
            sql = f"""SELECT ds_name FROM aaa_parameters WHERE ds_name LIKE '{search}%'"""
            df = pd.read_sql(sql, conn)
    except Exception as e:
        print(f"Error: {e}")
    return df

In [40]:
def get_data(connection, param, database, start_date, end_date):
    df = None
    try:
        # Read data
        with connection.cursor() as cursor:
            sql = f"""SELECT from_unixtime(timestamp/1000) as ts, value as {param} FROM {param} 
                  WHERE timestamp >= UNIX_TIMESTAMP('{start_date}') * 1000
                  AND timestamp < UNIX_TIMESTAMP('{end_date}') * 1000;"""
            # Using a median filter to remove spikes.  Modify as you wish.
            df = pd.read_sql(sql, conn, index_col='ts').resample(resample_interval).median()
            #df = pd.read_sql(sql, conn, index_col='ts')
    except Exception as e:
        print(f"Error: {e}")
    return df

In [41]:
df_names = get_names(conn)
with pd.option_context("display.max_rows", 1000): 
    print(df_names)

                    ds_name
0           temptrax1_count
1        temptrax1_interval
2          temptrax1_probe1
3         temptrax1_probe10
4         temptrax1_probe11
5         temptrax1_probe12
6         temptrax1_probe13
7         temptrax1_probe14
8         temptrax1_probe15
9         temptrax1_probe16
10         temptrax1_probe2
11         temptrax1_probe3
12         temptrax1_probe4
13         temptrax1_probe5
14         temptrax1_probe6
15         temptrax1_probe7
16         temptrax1_probe8
17         temptrax1_probe9
18      temptrax1_restarted
19  temptrax1_server_status
20      temptrax1_timestamp
21    temptrax1_unit_status
22         temptrax1_update
23          temptrax2_count
24       temptrax2_interval
25         temptrax2_probe1
26         temptrax2_probe2
27         temptrax2_probe3
28         temptrax2_probe4
29      temptrax2_restarted
30  temptrax2_server_status
31      temptrax2_timestamp
32    temptrax2_unit_status
33         temptrax2_update
34          temptrax

In [42]:
drop_list = [
    "cell_e_series_unit_status",
    "cell_e_series_count",
    "cell_e_series_ijb_raw",
    "cell_e_series_interval",
    "cell_e_series_restarted",
    "cell_e_series_server_status"
]
par_list = [
#    "cell_e_series_update",
#    "cell_e_series_timestamp",
#    "cell_e_series_crate_status",
    "cell_e_series_backplate_C",
    "cell_e_series_chamber_ambient_C",
    "cell_e_series_frontplate_C",
    "cell_e_series_in_front_of_primary_C",
    "cell_e_series_lower_plenum_C",
    "cell_e_series_midplate_C",
    "cell_e_series_outside_ambient_C",
    "yankee_temperature",
    "temptrax1_probe2",
    "temptrax1_probe3",
    "temptrax1_probe4",
    "temptrax1_probe6",
    "temptrax1_probe7",
    "temptrax1_probe8",
    "temptrax3_probe10",
    "temptrax3_probe11",
    "temptrax3_probe12"
]

In [43]:
df_all = None

for name in par_list:
    if debug:
        print(f"Getting {name} values...")
    df = get_data(conn, name, database, start_date, end_date)
    if df_all is None:
        df_all = df
    else:
        if df is not None:
            df_all = df_all.join(df, how='inner')

Getting cell_e_series_backplate_C values...
Getting cell_e_series_chamber_ambient_C values...
Getting cell_e_series_frontplate_C values...
Getting cell_e_series_in_front_of_primary_C values...
Getting cell_e_series_lower_plenum_C values...
Getting cell_e_series_midplate_C values...
Getting cell_e_series_outside_ambient_C values...
Getting yankee_temperature values...
Getting temptrax1_probe2 values...
Getting temptrax1_probe3 values...
Getting temptrax1_probe4 values...
Getting temptrax1_probe6 values...
Getting temptrax1_probe7 values...
Getting temptrax1_probe8 values...
Getting temptrax3_probe10 values...
Getting temptrax3_probe11 values...
Getting temptrax3_probe12 values...


In [44]:
df_all.tail()

,cell_e_series_backplate_C,cell_e_series_chamber_ambient_C,cell_e_series_frontplate_C,cell_e_series_in_front_of_primary_C,cell_e_series_lower_plenum_C,cell_e_series_midplate_C,cell_e_series_outside_ambient_C,yankee_temperature,temptrax1_probe2,temptrax1_probe3,temptrax1_probe4,temptrax1_probe6,temptrax1_probe7,temptrax1_probe8,temptrax3_probe10,temptrax3_probe11,temptrax3_probe12
ts,,,,,,,,,,,,,,,,,
2019-11-20 17:00:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.3940,11.6,9.9,9.6,10.6,5.75,8.5,10.8,9.8,9.6
2019-11-20 17:05:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.3120,11.6,9.9,9.6,10.6,5.80,8.5,10.8,9.8,9.6
2019-11-20 17:10:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.3035,11.6,9.9,9.6,10.6,6.00,8.5,10.8,9.8,9.6
2019-11-20 17:15:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.3820,11.6,9.9,9.6,10.6,6.10,8.5,10.8,9.8,9.6
2019-11-20 17:20:00,9.5527,10.6385,9.5911,9.8355,9.6118,9.4938,9.2789,8.3290,11.6,9.9,9.6,10.6,6.10,8.5,10.8,9.8,9.6


In [45]:
df_all.shape

(408881, 17)

In [46]:
df_all.to_csv("../halcoll/data/halcoll_temps.csv")

In [ ]:
# temptrax1 2, 3, 4, 6 (ne top, se top, se bottom, hub)
# temptrax3 10, 11, 12 (nw top, sw top, sw bottom)
# yankee_temperature

(.7e-6 ** 0.2) / (0.55e-6 ** 0.2)